# Selenium 사용해 멜론 TOP100 노래 추출하기

In [ ]:
# 1. 가상환경 생성
python -m venv selenium_env

# 2. 가상환경 활성화
# Windows
selenium_env\Scripts\activate
# macOS/Linux
source selenium_env/bin/activate

# 3. 필요한 패키지 설치
pip install selenium pandas

# 4. 가상환경 나가기
deactivate


In [26]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By


## 출력 표시 옵션

In [28]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## 사이트 접속

In [29]:
url = "https://www.melon.com/chart/index.htm?dayTime=2025101721"
driver = webdriver.Chrome()
driver.get(url)

## 특정 값 크롤링

In [33]:
import re 

# 멜론 차트 데이터를 저장할 DataFrame 생성
melon_chart_df = pd.DataFrame(columns=["제목", "아티스트", "수록곡"])
song_data_num = 0

# 멜론 차트에서 각 곡의 행(tr) 요소들 찾기
song_rows = driver.find_elements(By.CSS_SELECTOR, "#lst50")

for i, row in enumerate(song_rows):
    try:
        # 곡 제목 추출
        title_element = row.find_element(By.CSS_SELECTOR, "td:nth-child(6) > div > div > div.ellipsis.rank01 > span > a")
        song_title = title_element.text
        time.sleep(1)
        
        # 아티스트명 추출
        artist_element = row.find_element(By.CSS_SELECTOR, "td:nth-child(6) > div > div > div.ellipsis.rank02")
        artist_name = artist_element.text
        time.sleep(1)
        
        # 앨범명 클릭하여 상세 페이지로 이동
        album_element = row.find_element(By.CSS_SELECTOR, "td:nth-child(7) > div > div > div > a")
        album_element.click()
        time.sleep(3)
        
        # 앨범 상세 페이지에서 수록곡 정보 추출
        try:
            # 수록곡 리스트 추출
            track_elements = driver.find_elements(By.CSS_SELECTOR, "#frm > div > table > tbody > tr")
            track_list = []
            
            for track_row in track_elements[:10]:  # 최대 10곡까지만 수집
                try:
                    track_title = track_row.find_element(By.CSS_SELECTOR, "td:nth-child(4) > div > div > div:nth-child(1) > span > a").text
                    track_list.append(track_title)
                except:
                    continue
            
            # 수록곡들을 쉼표로 구분하여 하나의 문자열로 저장
            album_tracks = ", ".join(track_list) if track_list else "정보 없음"
            
        except:
            album_tracks = "정보 없음"
        
        # DataFrame에 데이터 저장
        melon_chart_df.loc[song_data_num] = [song_title, artist_name, album_tracks]
        song_data_num += 1
        
        # 차트 페이지로 돌아가기
        driver.back()
        time.sleep(2)
        
        print(f"{i+1}번째 곡 '{song_title}' 정보 수집 완료")
        
    except Exception as e:
        print(f"{i+1}번째 곡 정보 추출 실패: {e}")
        continue

print(f"총 {song_data_num}곡의 정보를 수집했습니다.")

1번째 곡 'Golden' 정보 수집 완료


KeyboardInterrupt: 

## 결과 출력

In [24]:
melon_chart_df

,제목,아티스트,수록곡
0,Golden,"HUNTR/X, EJAE, AUDREY NUNA, REI AMI, KPop Demo...","TAKEDOWN (JEONGYEON, JIHYO, CHAEYOUNG), How It..."
1,Drowning,WOODZ,"Deep Deep Sleep, Journey, Drowning, Busted, Wh..."
2,뛰어(JUMP),BLACKPINK,뛰어(JUMP)
3,Soda Pop,"KPop Demon Hunters Cast, Danny Chung, Saja Boy...","TAKEDOWN (JEONGYEON, JIHYO, CHAEYOUNG), How It..."
4,FAMOUS,ALLDAY PROJECT,"FAMOUS, WICKED"
5,시작의 아이 ❍,"박다혜, 마크툽 (MAKTUB)","시작의 아이 ❍, 시작의 아이 ❍ (inst.)"
6,Rich Man,aespa,"Rich Man, Drift, Bubble, Count On Me, Angel #4..."
7,어제보다 슬픈 오늘,우디 (Woody),"어제보다 슬픈 오늘, 어제보다 슬픈 오늘 (Inst.)"
8,XOXZ,IVE (아이브),"XOXZ, Wild Bird, Dear, My Feelings, GOTCHA (Ba..."
9,IRIS OUT,Kenshi Yonezu,IRIS OUT
